In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!ls "/content/drive/My Drive/jigsaw toxic"

sample_submission.csv  test.csv  test_labels.csv  train.csv


In [10]:
!ls "/content/drive/My Drive"

 Barrons-800.gdoc
 Classroom
'Colab Notebooks'
 Content-Based-Video-Search-TCS.zip
'datasets links'
 fra.txt
'[GigaCourse.com] Udemy - Deep Learning and Computer Vision A-Z™ OpenCV, SSD & GANs'
 glove.6B.100d.txt
 gre
'How to get started with Drive.pdf'
 Icons
 ipgm_project
'jigsaw toxic'
 magoos
'Mike And Dave Need Wedding Dates (2016) [1080p] [YTS.AG]'
'ml resume'
 project
'Research papers (final-year project).gsheet'
 TCS_PROJECT
 Untitled0.ipynb
'Untitled document.gdoc'
'Untitled spreadsheet.gsheet'
 videos
 YOLOv3-Object-Detection-with-OpenCV


In [6]:
# https://deeplearningcourses.com/c/deep-learning-advanced-nlp
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future


import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from sklearn.metrics import roc_auc_score

import keras.backend as K
# if len(K.tensorflow_backend._get_available_gpus()) > 0:
#   from keras.layers import CuDNNLSTM as LSTM
#   from keras.layers import CuDNNGRU as GRU

Using TensorFlow backend.


In [0]:
# some configuration
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 5


In [12]:
# load in pre-trained word vectors
print('Loading word vectors...')
word2vec = {}
with open("/content/drive/My Drive/glove.6B.100d.txt") as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
  for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))


Loading word vectors...
Found 400000 word vectors.


In [13]:
# prepare text samples and their labels
print('Loading in comments...')

train = pd.read_csv("/content/drive/My Drive/jigsaw toxic/train.csv")
sentences = train["comment_text"].fillna("DUMMY_VALUE").values
possible_labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
targets = train[possible_labels].values


Loading in comments...


In [17]:
sentences[1]

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [19]:
targets[0]

array([0, 0, 0, 0, 0, 0])

In [0]:
# convert the sentences (strings) into integers
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

In [21]:
# get word -> integer mapping
word2idx = tokenizer.word_index
print('Found %s unique tokens.' % len(word2idx))



Found 210337 unique tokens.


In [22]:
# pad sequences so that we get a N x T matrix
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (159571, 100)


In [25]:
# prepare embedding matrix
print('Filling pre-trained embeddings...')
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx.items():
  if i < MAX_VOCAB_SIZE:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
      # words not found in embedding index will be all zeros.
      embedding_matrix[i] = embedding_vector



Filling pre-trained embeddings...


In [0]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  input_length=MAX_SEQUENCE_LENGTH,
  trainable=False
)



In [27]:
# create an LSTM network with a single LSTM
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input_)
print(x.shape)
x = LSTM(15, return_sequences=True)(x)
print(x.shape)
# x = Bidirectional(LSTM(15, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
print(x.shape)
output = Dense(len(possible_labels), activation="sigmoid")(x)
print(output.shape)

(None, 100, 100)
(None, 100, 15)
(None, 15)
(None, 6)


In [0]:
model = Model(input_, output)
model.compile(
  loss='binary_crossentropy',
  optimizer=Adam(lr=0.01),
  metrics=['accuracy'],
)

In [29]:
print('Training model...')
r = model.fit(
  data,
  targets,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=VALIDATION_SPLIT
)

Training model...
Train on 127656 samples, validate on 31915 samples
Epoch 1/5
127656/127656 [==============================] - 66s 518us/step - loss: 0.0809 - accuracy: 0.9733 - val_loss: 0.0603 - val_accuracy: 0.9797
Epoch 2/5
127656/127656 [==============================] - 65s 511us/step - loss: 0.0563 - accuracy: 0.9805 - val_loss: 0.0573 - val_accuracy: 0.9800
Epoch 3/5
127656/127656 [==============================] - 66s 514us/step - loss: 0.0539 - accuracy: 0.9811 - val_loss: 0.0551 - val_accuracy: 0.9806
Epoch 4/5
127656/127656 [==============================] - 66s 520us/step - loss: 0.0524 - accuracy: 0.9814 - val_loss: 0.0542 - val_accuracy: 0.9807
Epoch 5/5
127656/127656 [==============================] - 66s 520us/step - loss: 0.0513 - accuracy: 0.9817 - val_loss: 0.0533 - val_accuracy: 0.9803


In [0]:
test = pd.read_csv("/content/drive/My Drive/jigsaw toxic/test.csv")


In [32]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [34]:
# prepare text samples and their labels
print('Loading test comments...')

train = pd.read_csv("/content/drive/My Drive/jigsaw toxic/test.csv")
sentences_test = train["comment_text"].fillna("DUMMY_VALUE").values


Loading test comments...


In [36]:
# convert the sentences (strings) into integers
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sentences_test)
sentences_test = tokenizer.texts_to_sequences(sentences_test)
sentences_test

[[1614,
  227,
  3383,
  812,
  8,
  56,
  82,
  884,
  337,
  16,
  3782,
  69,
  20,
  6,
  5,
  5515,
  6,
  1585,
  7,
  54,
  227,
  6234,
  1190,
  500,
  5001,
  5,
  93,
  6,
  2,
  2999,
  32,
  279,
  6,
  762,
  42,
  3383,
  812,
  8,
  35,
  4342,
  10,
  737,
  636,
  348,
  507,
  15299,
  9,
  171,
  15,
  158,
  5,
  15732,
  8,
  253,
  19272,
  44,
  2607,
  52,
  24,
  3,
  2225,
  154,
  1973,
  500,
  2110,
  93,
  219,
  144,
  486,
  84],
 [30, 1099, 1, 328, 8, 655, 17, 11, 8, 2688],
 [105, 6938, 15, 248],
 [22,
  6,
  19,
  3,
  148,
  162,
  28,
  1,
  118,
  1,
  114,
  7,
  1553,
  24,
  1,
  387,
  651,
  7,
  39,
  77,
  669,
  1,
  118,
  3068,
  1553,
  7,
  1373,
  1314,
  1,
  114,
  431,
  133,
  25,
  147,
  6,
  13,
  21,
  411],
 [7, 57, 10184, 81, 85, 28, 38],
 [147, 6, 13, 1159, 7, 61, 104, 1209, 4, 6, 5, 43, 14, 447, 196, 138],
 [53,
  36,
  14,
  161,
  541,
  2,
  29,
  112,
  172,
  18,
  432,
  247,
  5,
  1823,
  4897,
  22,
  6,
  43,
  44

In [37]:
# get word -> integer mapping
word2idx = tokenizer.word_index
print('Found %s unique tokens.' % len(word2idx))


Found 273192 unique tokens.


In [38]:
# pad sequences so that we get a N x T matrix
data = pad_sequences(sentences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (153164, 100)


In [40]:
y_pred = model.predict(data)
y_pred

array([[6.7546880e-01, 5.1564574e-03, 3.6151892e-01, 1.7943382e-03,
        3.0095637e-01, 1.1265010e-02],
       [9.8872018e-01, 1.3262379e-01, 9.6831930e-01, 9.6760988e-03,
        8.7532985e-01, 4.8052073e-02],
       [1.2930632e-02, 3.9926171e-04, 3.8850605e-03, 1.2108684e-03,
        5.5120885e-03, 8.8977814e-04],
       ...,
       [9.9188089e-04, 1.5937543e-05, 3.8611889e-04, 2.5477184e-05,
        4.2501092e-04, 4.4110639e-05],
       [1.4804125e-02, 2.1788478e-04, 1.6979873e-03, 9.4795227e-04,
        3.4417808e-03, 3.9759278e-04],
       [9.2720687e-03, 9.9299883e-05, 2.2487044e-03, 1.5905499e-04,
        3.4956038e-03, 2.7205348e-03]], dtype=float32)

In [41]:
y_pred[0]

array([0.6754688 , 0.00515646, 0.36151892, 0.00179434, 0.30095637,
       0.01126501], dtype=float32)

In [43]:
id = pd.read_csv('/content/drive/My Drive/jigsaw toxic/test_labels.csv')
id.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [44]:
output = pd.DataFrame({'Comment_Id': id.id, 'toxic': y_pred[:,0], 'severe_toxic': y_pred[:,1], 'obscene': y_pred[:,2], 'threat': y_pred[:,3], 'insult': y_pred[:,4], 'identity_hate': y_pred[:,5]})
output.head()
# output.to_csv('my_submission1.csv', index=False)
# print("Your submission was successfully saved!")

,Comment_Id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.675469,0.005156,0.361519,0.001794,0.300956,0.011265
1,0000247867823ef7,0.988720,0.132624,0.968319,0.009676,0.875330,0.048052
2,00013b17ad220c46,0.012931,0.000399,0.003885,0.001211,0.005512,0.000890
3,00017563c3f7919a,0.003592,0.000063,0.003280,0.000044,0.001379,0.000179
4,00017695ad8997eb,0.006672,0.000175,0.002303,0.000431,0.002702,0.000359


In [45]:
output.shape

(153164, 7)

In [46]:
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
